In [1]:
import pandas as pd
import ast
from urllib.parse import urlparse, parse_qs
from os.path import commonprefix

""" New sentences """
new_sentences = pd.read_excel("new_sentences.xlsx")

""" New transcripts """
def parse_id(metadata):
    url = ast.literal_eval(metadata)["id"]
    query_params = parse_qs(urlparse(url).query)
    return query_params["v"][0]

new_transcripts = new_sentences.groupby(by=lambda i: parse_id(new_sentences.Metadata.loc[i])).agg({
    "Name": lambda names: commonprefix(list(names))[:-1],
    "Content": "\n".join,
    "Tags": "first",
    "Metadata": "first",
}).reset_index(drop=True)

""" New controls """
new_controls = pd.DataFrame({
    "Name": new_transcripts.Name,
    "Content": new_transcripts.Name + new_transcripts.Metadata.apply(lambda metadata: ast.literal_eval(metadata)["description"]),
    "Tags": new_transcripts.Tags,
    "Metadata": new_transcripts.Metadata
})

new_sentences["Tags"] = new_sentences["Tags"].apply(lambda tags: "SENTENCE_LEVEL, " + tags)
new_transcripts["Tags"] = new_transcripts["Tags"].apply(lambda tags: "TRANSCRIPT_LEVEL, " + tags)
new_controls["Tags"] = new_controls["Tags"].apply(lambda tags: "CONTROL, " + tags)

# Merge into current library
lib = pd.read_excel("Backup-03-03-2023.xlsx")
lib = pd.concat([lib, new_sentences, new_transcripts, new_controls], ignore_index=True)

# Remove university-level playlist
videos_to_remove = ["ttUWTu3jusA", "6V13YiuJ-nc", "PY4k_BlAlSU", "K6mHqvq8G0w", "byjdCpCNsBg",
                    "AFiOJfQDs6E", "UyTKlAs9I-M", "0c82055Tvwg", "DZu5M0g5IbQ", "DZu5M0g5IbQ",
                    "SVNB-jCV7gM", "LDGfWynJrGg", "bC3BUHhxzFQ", "RsqeDCDT6Kg", "kY2OKQfasAs",
                    "qPHdGX8Lf54", "AoA2T3dPuwk", "COjis7G-aQY", "hLk2ujAk1pk", "O-UheXfQqNE",
                    "D2dZ-z2e6K4", "o3ENskH6fKo", "8NI38LxYvwY", "E5pl7b1OChE", "ok05H5I2y9A",
                    "bcn6WN0iv_s", "gzb94vOW2i8", "Y8g4S-PfeDU", "o4Y2A-CSQwg", "58W1oArE8p8",
                    "8Snag_AgNyU"]

lib = lib[lib["Metadata"].apply(lambda metadata: parse_id(metadata) not in videos_to_remove)]
lib.to_excel("enriched_lib.xlsx")


C:\Users\TomKaminski\Documents\AIEDN\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
